In [1]:
import os
import pandas as pd
import rasterio
import geopandas as gpd
import numpy as np
from rasterstats import zonal_stats
from shapely.geometry import Point, box

## Define all paths for the tif files

In [6]:
main_path_mac = "/Users/jack/Library/CloudStorage/GoogleDrive-limjackailjk@gmail.com/My Drive/Solar PV Lab/NIMBY Project/Jenny's Regression/Data Sources/"
main_path_win = "G:/My Drive/Solar PV Lab/NIMBY Project/Jenny's Regression/Data Sources/"

GHI= "GHI-09188ce2.tif"
protected_land = "Protected_Land-5745a356.tif"
habitat= "Habitat-32079c87.tif"
slope= "slope_only-2c1658fa.tif"
popl_dens= "Popl_Density-714f0a64.tif"
Substation = "distance_to_substation_only-f02c9129.tif"
land_cover="Land_Cover-8a2691e6.tif"
# Define the TIF file paths as a list
tif_paths = [GHI, protected_land, habitat, slope, popl_dens, Substation, land_cover]
tif_paths_full = [main_path_mac + path for path in tif_paths]
tif_paths_full

["/Users/jack/Library/CloudStorage/GoogleDrive-limjackailjk@gmail.com/My Drive/Solar PV Lab/NIMBY Project/Jenny's Regression/Data Sources/GHI-09188ce2.tif",
 "/Users/jack/Library/CloudStorage/GoogleDrive-limjackailjk@gmail.com/My Drive/Solar PV Lab/NIMBY Project/Jenny's Regression/Data Sources/Protected_Land-5745a356.tif",
 "/Users/jack/Library/CloudStorage/GoogleDrive-limjackailjk@gmail.com/My Drive/Solar PV Lab/NIMBY Project/Jenny's Regression/Data Sources/Habitat-32079c87.tif",
 "/Users/jack/Library/CloudStorage/GoogleDrive-limjackailjk@gmail.com/My Drive/Solar PV Lab/NIMBY Project/Jenny's Regression/Data Sources/slope_only-2c1658fa.tif",
 "/Users/jack/Library/CloudStorage/GoogleDrive-limjackailjk@gmail.com/My Drive/Solar PV Lab/NIMBY Project/Jenny's Regression/Data Sources/Popl_Density-714f0a64.tif",
 "/Users/jack/Library/CloudStorage/GoogleDrive-limjackailjk@gmail.com/My Drive/Solar PV Lab/NIMBY Project/Jenny's Regression/Data Sources/distance_to_substation_only-f02c9129.tif",
 "/

## Load the Counties Bounding Boxes and get the state and county names

In [7]:
# block_group_bounding_boxes_win = pd.read_csv("G:/My Drive/Solar PV Lab/NIMBY Project/Solar NIMBY Final/Solar-NIMBY/data cleaning/data_bg/bounding_box_bg_full.csv", dtype={'GEOID': str, 'STATEFP': str, 'COUNTYFP': str, 'TRACTCE': str, 'BLKGRPCE': str})

# bg_bb_full_win = gpd.read_file("C:/Users/limja/Downloads/cb_2023_us_bg_500k/cb_2023_us_bg_500k.shp")[['GEOID', 'geometry']] # Not in repo due to size

block_group_bounding_boxes_mac = pd.read_csv("/Users/jack/Library/CloudStorage/GoogleDrive-limjackailjk@gmail.com/My Drive/Solar PV Lab/NIMBY Project/Solar NIMBY Final/Solar-NIMBY/data cleaning/data_bg/bounding_box_bg_full.csv", dtype={'GEOID': str, 'STATEFP': str, 'COUNTYFP': str, 'TRACTCE': str, 'BLKGRPCE': str})

bg_bb_full_mac = gpd.read_file("/Users/jack/Downloads/US Boundary 500k/cb_2023_us_bg_500k.shp")[['GEOID', 'geometry']] # Not in repo due to size

# Merge the bounding boxes with the block group geometries
block_group_bounding_boxes = bg_bb_full_mac.merge(block_group_bounding_boxes_mac, on='GEOID', how='left')

block_group_bounding_boxes.head()

,GEOID,geometry,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,State,County Name,area km2,area mi2
0,011130303001,"POLYGON ((-85.00365 32.47885, -85.00133 32.480...",01,113,030300,1,Alabama,Russell,0.802873,0.309991
1,040159534062,"POLYGON ((-114.32649 34.43788, -114.31824 34.4...",04,015,953406,2,Arizona,Mohave,592.457766,228.749128
2,040019450021,"POLYGON ((-109.32335 35.54182, -109.31495 35.5...",04,001,945002,1,Arizona,Apache,608.287674,234.861087
3,040270003012,"POLYGON ((-114.66727 32.72505, -114.66541 32.7...",04,027,000301,2,Arizona,Yuma,2.098048,0.810061
4,050070208033,"POLYGON ((-94.27908 36.48882, -94.27656 36.491...",05,007,020803,3,Arkansas,Benton,3.594376,1.387796


In [ ]:
block_group_bounding_boxes

In [ ]:
# Load FIPS and county data for names
county_bounding_boxes = pd.read_csv("/Users/jack/Library/CloudStorage/GoogleDrive-limjackailjk@gmail.com/My Drive/Solar PV Lab/NIMBY Project/Solar NIMBY Final/Solar-NIMBY/data cleaning/data/county_bounding_boxes.csv", dtype={"FIPS State": str, "FIPS County": str})

county_bounding_boxes.head()

In [ ]:
# load the full county boxes 
county_bounding_boxes_full = gpd.read_file("/Users/jack/Library/CloudStorage/GoogleDrive-limjackailjk@gmail.com/My Drive/Solar PV Lab/NIMBY Project/Solar NIMBY Final/Solar-NIMBY/county_box/US County Boundary 2018/cb_2018_us_county_500k.shp", dtype={'STATEFP': str}).rename(columns={"STATEFP": "FIPS State", "COUNTYFP": "FIPS County"})
county_bounding_boxes_full.head()

In [ ]:
# Merge the Files
county_bounding_boxes_full = county_bounding_boxes_full.merge(county_bounding_boxes, on=["FIPS State", "FIPS County"], how="left")

county_bounding_boxes_full.head()

In [5]:
col_names = ["GHI", "Protected_Land", "Habitat", "Slope", "Population_Density", "Distance_to_Substation", "Land_Cover"]

def calculate_zonal_stats(tif_path, geodataframe, nodata_value ):
    with rasterio.open(tif_path) as src:
        affine = src.transform
        array = src.read(1)  # Read the first band
        array = np.where(np.isnan(array), nodata_value, array)  # Replace NaNs with nodata_value
        # Debugging: Check raster data and affine transformation
        print(f"Raster data shape: {array.shape}")
        print(f"Affine transformation: {affine}")
        # Check the CRS of the raster
        raster_crs = src.crs
        print(f"Raster CRS: {raster_crs}")
        if geodataframe.crs != raster_crs:
            geodataframe = geodataframe.to_crs(raster_crs)
            
        # filter out values that are less than 100
        array = np.where(array > 101, nodata_value, array)

    # Calculate zonal statistics
    stats = zonal_stats(geodataframe, array, affine=affine, stats="mean", nodata=nodata_value, all_touched=True)
    print(stats)
    # Extract mean values and add to GeoDataFrame
    mean_values = [stat['mean'] for stat in stats]
    return mean_values

def process_tif_files(tif_filepaths, bounding_box, nodata_value=-9999,bg=False):
    x = bounding_box.copy()

    # Initialize results DataFrame
    results = pd.DataFrame(index=x.index, columns=col_names)

    for tif_path, col_name in zip(tif_filepaths, col_names):
        print(f"Processing {tif_path} for {col_name}")

        # Calculate mean values using zonal stats
        mean_values = calculate_zonal_stats(tif_path, x, nodata_value)

        # Update results DataFrame
        results[col_name] = mean_values

    # Add county and state information
    results["County Name"] = bounding_box["County Name"]
    results["State"] = bounding_box["State"]
    if bg:
        results["GEOID"] = bounding_box["GEOID"]
        results["TRACTCE"] = bounding_box["TRACTCE"]
        results["BLKGRPCE"] = bounding_box["BLKGRPCE"]

    return results

In [ ]:
# project the bounding box to the same crs as the tif files
county_bounding_boxes_full = county_bounding_boxes_full.to_crs("EPSG:4326")
technoecon_suitability_scores = process_tif_files(tif_paths_full, county_bounding_boxes_full[['geometry', 'County Name', 'State']], nodata_value=np.nan)

In [ ]:
block_group_bounding_boxes_4326 = block_group_bounding_boxes.to_crs("EPSG:4326")
block_group_suitability_scores = process_tif_files(tif_paths_full, block_group_bounding_boxes_4326[['geometry', 'GEOID', 'County Name', 'State', 'TRACTCE', "BLKGRPCE"]], nodata_value=np.nan, bg=True)

In [36]:
mapper = {
    '110': 'Hartford', '190': 'Fairfield', '170': 'Litchfield', 
    '140': 'Middlefield', '120': 'New Haven', '130': 'Tolland',
    '160': 'Windham', '180': 'New London', '150': 'New London'
}

In [49]:
def fix_data_Connecticut(series):
    
    if series['GEOID'][:2] == '09':
        series['State'] = 'Connecticut'
        series['County Name'] = mapper[series['GEOID'][2:5]]
        series['TRACTCE'] = series['GEOID'][5:11]
        series['BLKGRPCE'] = series['GEOID'][11:]

    return series

In [50]:
block_group_suitability_scores = block_group_suitability_scores.apply(fix_data_Connecticut, axis=1)

In [ ]:
block_group_suitability_scores.head()

In [2]:
block_group_suitability_scores.to_csv('block_group_suitability_scores.csv', index=False)

NameError: name 'block_group_suitability_scores' is not defined

In [44]:
block_group_suitability_scores = pd.read_csv("/Users/jack/Library/CloudStorage/GoogleDrive-limjackailjk@gmail.com/My Drive/Solar PV Lab/NIMBY Project/Solar NIMBY Final/Solar-NIMBY/regression/cleaned data bg/block_group_suitability_scores.csv",
                                             dtype={'GEOID': str, 'TRACTCE': str, 'BLKGRPCE': str})

FIPS = pd.read_csv("/Users/jack/Library/CloudStorage/GoogleDrive-limjackailjk@gmail.com/My Drive/Solar PV Lab/NIMBY Project/Solar NIMBY Final/Solar-NIMBY/data cleaning/data/FIPS.csv", dtype={'FIPS State': str, 'FIPS County': str})

In [45]:
state_dict = FIPS.set_index('FIPS State')['State'].to_dict()

# County dict for mapping but requires to match the FIPS State as well
county_dict = FIPS.set_index(['FIPS State', 'FIPS County'])['County Name'].to_dict()

In [46]:
def fix_tract(series):
    if series['GEOID'] != np.nan:
        if series['GEOID'][:2] == '02':
            series['State'] = 'Alaska'
        elif series['GEOID'][:2] == '15':
            series['State'] = 'Hawaii'
        else:
            try:
                series['State'] = state_dict[series['GEOID'][:2]]
            except:
                series['State'] = np.nan
        series['TRACTCE'] = series['GEOID'][5:11]
        series['BLKGRPCE'] = series['GEOID'][11:]
        try:
            series['County Name'] = county_dict[(series['GEOID'][:2], series['GEOID'][2:5])]
        except:
            series['County Name'] = np.nan
        
    return series

In [47]:
block_group_suitability_scores = block_group_suitability_scores.apply(fix_tract, axis=1)
block_group_suitability_scores = block_group_suitability_scores.dropna(subset=['State', 'County Name'])

# Check to see if there are no statistics for certain geoid

In [ ]:
list_in = ['040190043371', '081070004003'] # should be in
list_out = ['020900001002', '150030038023'] # should be out

In [ ]:
# Open the raster file
GHI_raster = rasterio.open(tif_paths_full[0])

# Get the bounding box of the raster as a Shapely Polygon
raster_bounds = box(*GHI_raster.bounds)
raster_crs = GHI_raster.crs

bb = block_group_bounding_boxes.to_crs(raster_crs)

# Define GEOID and get its bounding box as a Shapely Polygon from the DataFrame
for GEOID in list_in:
    GEOID_polygon = bb[bb['GEOID'] == GEOID].geometry.values[0]

    # Check if the GEOID polygon is within the raster bounds
    within_bounds = GEOID_polygon.within(raster_bounds)

    print("Is the GEOID bounding box within raster bounds?", within_bounds)

for GEOID in list_out:
    GEOID_polygon = bb[bb['GEOID'] == GEOID].geometry.values[0]

    # Check if the GEOID polygon is within the raster bounds
    within_bounds = GEOID_polygon.within(raster_bounds)

    print("Is the GEOID bounding box within raster bounds?", within_bounds)